In [ ]:
# --- Inputs ---
import time, io, csv, json, boto3, botocore, sagemaker, sys
from sagemaker import get_execution_role

region        = boto3.Session().region_name
bucket        = "your-s3-bucket"
train_key     = "data/your-training.csv"     # CSV with header (includes target)
target_col    = "target"
problem_type  = "MulticlassClassification"   # or BinaryClassification / Regression
objective     = "F1Macro"                    # pick an appropriate metric
mode          = "AUTO"                       # AUTO | ENSEMBLING | HYPERPARAMETER_TUNING
instance_type = "ml.m5.large"
endpoint_name = "autopilot-poc-endpoint"

s3 = boto3.client("s3", region_name=region)
sm = boto3.client("sagemaker", region_name=region)
rt = boto3.client("sagemaker-runtime", region_name=region)

# get_execution_role() works in SageMaker environments; fall back otherwise
try:
    role = get_execution_role()
except Exception:
    role = "arn:aws:iam::<ACCOUNT_ID>:role/<SageMakerExecutionRole>"

job_name        = f"autopilot-{int(time.time())}"
s3_train_path   = f"s3://{bucket}/{train_key}"
s3_output_path  = f"s3://{bucket}/autopilot-output/{job_name}"
feature_spec_key = f"{job_name}/features.json"

# --- Read CSV header safely ---
obj = s3.get_object(Bucket=bucket, Key=train_key)
header = next(csv.reader(io.TextIOWrapper(obj["Body"], encoding="utf-8")))
if target_col not in header:
    raise ValueError(f"Target column '{target_col}' not found in header: {header}")

# Build feature list in deterministic order (header minus target)
feature_columns = [c for c in header if c != target_col]

# --- Feature spec (no TargetAttributeName here) ---
feature_spec = {
    "FeatureAttributeNames": feature_columns,
    # Optional: "FeatureDataTypes": [{"FeatureName":"colA","FeatureDataType":"String"}, ...]
}
s3.put_object(Bucket=bucket, Key=feature_spec_key, Body=json.dumps(feature_spec).encode("utf-8"))
feature_spec_uri = f"s3://{bucket}/{feature_spec_key}"

# --- Launch AutoML V2 job (explicit objective + mode + completion caps) ---
sm.create_auto_ml_job_v2(
    AutoMLJobName=job_name,
    AutoMLJobInputDataConfig=[{
        "ChannelType": "training",
        "ContentType": "text/csv;header=present",
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": s3_train_path}},
        "TargetAttributeName": target_col
    }],
    AutoMLJobOutputDataConfig={"S3OutputPath": s3_output_path},
    RoleArn=role,
    AutoMLJobObjective={"MetricName": objective},
    AutoMLProblemTypeConfig={
        "TabularJobConfig": {
            "ProblemType": problem_type,
            "Mode": mode,
            "FeatureSpecificationS3Uri": feature_spec_uri,
            "CompletionCriteria": {
                "MaxCandidates": 3,                          # keep tiny for PoC
                "MaxRuntimePerTrainingJobInSeconds": 1800,   # 30 min/job
                "MaxAutoMLJobRuntimeInSeconds": 7200         # 2 hours total
            }
        }
    }
)
print("Started:", job_name)

# --- Poll with simple backoff ---
sleep = 30
deadline = time.time() + 3*3600
while True:
    d = sm.describe_auto_ml_job_v2(AutoMLJobName=job_name)
    st = d["AutoMLJobStatus"]; sec = d.get("AutoMLJobSecondaryStatus")
    print("Status:", st, "-", sec)
    if st in ("Completed", "Failed", "Stopped"):
        break
    if time.time() > deadline:
        raise TimeoutError("AutoML job exceeded local wait deadline.")
    time.sleep(sleep)
    if sleep < 120:
        sleep += 10  # gentle backoff

if st != "Completed":
    raise RuntimeError(f"AutoML V2 failed: {st} ({sec})")

# --- Best candidate & deploy (multi-container aware) ---
best = d["BestCandidate"]
model_name = f"{job_name}-model"
cfg_name   = f"{job_name}-cfg"

sm.create_model(
    ModelName=model_name,
    Containers=best["InferenceContainers"],  # Handles pipelines automatically
    ExecutionRoleArn=role
    # Optionally: EnableNetworkIsolation=True, VpcConfig={...}
)

sm.create_endpoint_config(
    EndpointConfigName=cfg_name,
    ProductionVariants=[{
        "VariantName": "AllTraffic",
        "ModelName": model_name,
        "InstanceType": instance_type,
        "InitialInstanceCount": 1
    }]
)

# Create-or-update endpoint (narrow exception to ResourceNotFound)
try:
    sm.describe_endpoint(EndpointName=endpoint_name)
    print("Updating endpoint:", endpoint_name)
    sm.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=cfg_name)
except sm.exceptions.ResourceNotFound:
    print("Creating endpoint:", endpoint_name)
    sm.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=cfg_name)

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)
print("Endpoint InService:", endpoint_name)

# --- Quick test: build payload in *feature_columns* order ---
obj2 = s3.get_object(Bucket=bucket, Key=train_key)
lines = obj2["Body"].read().decode("utf-8").splitlines()
if len(lines) > 1:
    sample = dict(zip(header, next(csv.reader([lines[1]]))))
    payload_row = [sample.get(col, "") for col in feature_columns]  # exact order
    payload = ",".join(payload_row) + "\n"
    resp = rt.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="text/csv",
        Body=payload.encode("utf-8")
    )
    print("Sample prediction:", resp["Body"].read().decode("utf-8"))
